Code based in https://ai.google.dev/gemma/docs/core/huggingface_text_finetune_qlora

# Usando QLoRA para fine-tuning Eficiente

Este notebook mostra o uso do Quantized Low-Rank Adaptation (QLoRA), um método  para o fine-tuning eficiente de LLMs, que reduz os requisitos computacionais enquanto mantém um alto desempenho. No QLoRA, o modelo pré-treinado é quantizado para 4 bits e seus pesos são congelados. Em seguida, camadas de adaptação treináveis (LoRA) são adicionadas, e apenas essas camadas são treinadas. Posteriormente, os pesos das camadas adaptadoras podem ser mesclados ao modelo base ou mantidos como um adaptador separado.

In [1]:
!pip install datasets --quiet
!pip install peft --quiet
!pip install trl --quiet
!pip install bitsandbytes --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
!nvidia-smi

Sat Apr  5 22:13:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import numpy as np
import pandas as pd
import os
import sys
import os
from tqdm import tqdm
import json
import torch
sys.path.append(".")
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from accelerate import Accelerator
from torch.utils.data import DataLoader
from transformers import Trainer
import seaborn as sns
from matplotlib import pyplot as plt
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
import time

import warnings
warnings.filterwarnings("ignore")

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Vamos utilizar, como anteriormente o modelo da família SmolLMs com 360M de parâmetros.

In [6]:
model_name = 'HuggingFaceTB/SmolLM-1.7B'

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

E para testar o Fine-Tuning vamos utilizar um dataset cujo o objetivo é transformar uma requisição em linguagem natural em um código de consulta de banco de dados SQL.

In [8]:
def sql_format_func(example):
    example["Text"] = f"### SQL Prompt:\n{example['sql_prompt']}\n### Response:\n{example['sql']}"
    return example

In [9]:
dataset = load_dataset("gretelai/synthetic_text_to_sql")
dataset = dataset.map(lambda x: sql_format_func(x))

README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [10]:
#tokenizing dataset
dataset = dataset.map(lambda samples: tokenizer(samples['Text']), batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5851 [00:00<?, ? examples/s]

Inicialmente definiremos o LoRA e os parâmetros de treinamento como anteriormente, aplicados nas camadas de atenção do modelo e com um rank baixo.

In [11]:
peft_config_q = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj"],
    task_type="CAUSAL_LM"
)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules=["q_proj", "k_proj", "v_proj"],
    task_type="CAUSAL_LM"
)

Este código configura os parâmetros de treinamento para um fine-tuning utilizando QLoRA com a biblioteca trl. A configuração é definida através da classe SFTConfig, que estabelece diversas opções para o treinamento do modelo. Aqui está uma definição detalhada de cada parâmetro:

In [12]:
max_steps = 20

args = SFTConfig(
    output_dir="output/lora",         # directory to save and repository id
    max_seq_length=512,                     # max sequence length for model and packing of the dataset
    packing=True,                           # Groups multiple samples in the dataset into a single sequence
    max_steps=max_steps,                    # max number of training steps
    per_device_train_batch_size=2,          # batch size per device during training
    gradient_accumulation_steps=4,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="paged_adamw_8bit",               # paged adamw optimizer for training
    logging_steps=int(max_steps/10),        # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    dataset_kwargs={
        "add_special_tokens": False, # We template with special tokens
        "append_concat_token": True, # Add EOS token as separator token between examples
    },
    report_to='none',
    dataset_text_field = 'Text',
)

In [13]:
max_steps = 20

q_args = SFTConfig(
    output_dir="output/Qlora",         # directory to save and repository id
    max_seq_length=512,                     # max sequence length for model and packing of the dataset
    packing=True,                           # Groups multiple samples in the dataset into a single sequence
    max_steps=max_steps,                    # max number of training steps
    per_device_train_batch_size=2,          # batch size per device during training
    gradient_accumulation_steps=4,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="paged_adamw_8bit",               # paged adamw optimizer for training
    logging_steps=int(max_steps/10),        # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    dataset_kwargs={
        "add_special_tokens": False, # We template with special tokens
        "append_concat_token": True, # Add EOS token as separator token between examples
    },
    report_to='none',
    dataset_text_field = 'Text',
)

Este código define a configuração de quantização para carregar um modelo pré-treinado em 4 bits, essencial para o funcionamento eficiente do **QLoRA**. A biblioteca `bitsandbytes`, é utilizada para definir o parâmetro `load_in_4bit=True` ativa a quantização de 4 bits, enquanto `bnb_4bit_use_double_quant=True` aplica uma quantização dupla para otimizar ainda mais o uso de memória.

O tipo de quantização `nf4` (Normal Float 4) é escolhido, pois, conforme demonstrado no artigo do QLoRA, ele melhora a representatividade dos pesos do modelo em comparação com a quantização tradicional. Além disso, `bnb_4bit_compute_dtype=torch.bfloat16` e `bnb_4bit_quant_storage=torch.bfloat16` especificam que os cálculos e o armazenamento da quantização serão feitos no formato bfloat16, que oferece uma boa precisão com menor consumo de memória.

In [14]:
from peft import prepare_model_for_kbit_training

quant_config = BitsAndBytesConfig(
    #load_in_8bit=True,
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_storage=torch.bfloat16
)

model_q = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quant_config)
model_q.gradient_checkpointing_enable()
model_q = prepare_model_for_kbit_training(model_q)
model_q = get_peft_model(model_q, peft_config_q)
model_q.print_trainable_parameters()

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/17.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

trainable params: 2,359,296 || all params: 1,713,735,680 || trainable%: 0.1377


In [15]:
response_temp = '### Response:\n'
response_temp_ids = tokenizer(response_temp)['input_ids']
data_collator = DataCollatorForCompletionOnlyLM(response_temp_ids, tokenizer = tokenizer)

model_q = model_q.to('cuda')
q_trainer = Trainer(
    model=model_q,
    args=q_args,
    train_dataset=dataset["train"],
    data_collator=data_collator,
    #peft_config=peft_config,
)

# Create Trainer object
# trainer = SFTTrainer(
#     model=model_q,
#     args=args,
#     train_dataset=dataset["train"],
#     processing_class=tokenizer
# )

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


E a seguir vamos medir o tempo para rodar o ajuste utilizando o modelo quantizado pela técnica do QLoRA. Além disso vamos medir também a memória gasta durante o treinamento do modelo.

In [16]:
start = time.time()
#model_q.train()
#model_q.enable_input_require_grads()
q_trainer.train()
print(f"Training time: {time.time()-start} seconds")
print(f"Peak memory usage: {torch.cuda.max_memory_allocated() / 1024**3} GB")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
2,0.815300
4,0.947700
6,0.931600
8,1.078300
10,1.033900
12,0.832100
14,0.786400
16,0.851500
18,0.591600
20,0.673800


Training time: 98.86228656768799 seconds
Peak memory usage: 1.6321258544921875 GB


In [17]:
del model_q, q_trainer

In [18]:
#limpando a memória de GPU para fazer novas medições
torch.cuda.reset_peak_memory_stats()
torch.cuda.empty_cache()

Agora, carregaremos o modelo tradicional, sem quantizações impostas pela técnica do QLoRA para fins de comparação. Além disso todas as configurações serão as mesmas das utilizadas anteriormente pelo parâmetro SFTConfig.

In [19]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model.gradient_checkpointing_enable()
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

response_temp = '### Response:\n'
response_temp_ids = tokenizer(response_temp)['input_ids']
data_collator = DataCollatorForCompletionOnlyLM(response_temp_ids, tokenizer = tokenizer)

model = model.to("cuda")

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    data_collator=data_collator,
    #peft_config=peft_config,
    #processing_class=tokenizer
)

# trainer = SFTTrainer(
#     model=model,
#     args=args,
#     train_dataset=dataset["train"],
#     #peft_config=peft_config,
#     processing_class=tokenizer
# )

start = time.time()
trainer.train()

print(f"Training time: {time.time()-start} seconds")
print(f"Peak memory usage: {torch.cuda.max_memory_allocated() / 1024**3} GB")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 2,359,296 || all params: 1,713,735,680 || trainable%: 0.1377


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
2,0.721700
4,0.846100
6,0.814900
8,0.999800
10,0.952600
12,0.735400
14,0.730900
16,0.802600
18,0.578600
20,0.612000


Training time: 79.43404817581177 seconds
Peak memory usage: 7.558160781860352 GB


In [20]:
del model, trainer

In [21]:
torch.cuda.reset_peak_memory_stats()
torch.cuda.empty_cache()

Os resultados mostram que o QLoRA alcançou uma loss final parecida com à do LoRA padrão, demonstrando que a quantização para 4 bits não afetou significativamente o desempenho do modelo. A principal vantagem do QLoRA foi a redução significativa no uso de memória, consumindo apenas 1.84 GB de VRAM, enquanto o LoRA padrão exigiu 6.8 GB. Isso confirma que o QLoRA permite o fine-tuning eficiente de grandes modelos em hardware mais limitado, mantendo praticamente o mesmo desempenho do LoRA tradicional.